### Part 1: Data Collection
* I have selected the JSON URL and load it to the local using python
* I stored the the JSON file as "todos_data.json" in the "data" directory

In [ ]:
import os
import requests
import json

def fetch_data(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an exception for bad status codes
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Failed to fetch data from the URL: {e}")
        return None
    except json.JSONDecodeError as e:
        print(f"Failed to decode JSON data: {e}")
        return None

def save_data(data, directory, file_name):
    try:
        # Create directory if it doesn't exist
        if not os.path.exists(directory):
            os.makedirs(directory)

        file_path = os.path.join(directory, file_name)
        with open(file_path, 'w') as f:
            json.dump(data, f)
        print(f"Data saved to {file_path}")
    except IOError as e:
        print(f"Failed to write data to file: {e}")


if __name__ == "__main__":
    url = "https://jsonplaceholder.typicode.com/todos"
    file_name = "todos_data.json"
    directory = "data"
    
    todos_data = fetch_data(url)
    if todos_data:
        save_data(todos_data, directory, file_name)


### Part 2: Data Transformation
Data Transformation is usually seperated into 2 parts -- Data Validation & Normalization

#### Data Validation includes:
1. Handle missing value: Missing values can lead to data inconsistency and affect the results of queries.
2. Handle duplicate values: Data Duplication can violate constraints like primary keys, unique keys, and foreign keys
3. Data Type Conversion: Ensures data of each columns is stored in the correct data type, and these data types should match the later corresponding columns in MSSQL database.


### Normalization includes:
1. Split data into Multiple Tables: to minimize redundancy and improve data integrity, got to determine the table structures.
2. Determine Primary keys: Determine the primary keys of each table that is unique to each row in the table.
3. Determine Foreign Keys: After splittting data into multiple table, we need to identify relationships between these tables and use foreign keys to enforce referential integrity.

In [1]:
import pandas as pd

# Load JSON data into DataFrame, take a look at te first few rows to verify the data collected is fine
df = pd.read_json("./data/todos_data.json")
print(df.head())

   userId  id                                              title  completed
0       1   1                                 delectus aut autem      False
1       1   2                 quis ut nam facilis et officia qui      False
2       1   3                                fugiat veniam minus      False
3       1   4                                   et porro tempora       True
4       1   5  laboriosam mollitia et enim quasi adipisci qui...      False


/var/folders/hx/_m156f0s2h7fhh_t5ffqzm680000gn/T/ipykernel_56866/3597807945.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [ ]:
# check if there is any missing columns
print("Checking Missing Columns:")
print(df.isna().sum())
print()

# check if there is any duplications:
print("Checking Duplicated Rows:")
print(df.duplicated().sum())
print()

# check data type 
print("Checking Data Types:")
print(df.dtypes)
print()


##### Data Validation Result: 
1. No missing values
2. No duplicated rows
3. For the data type, 'title' is in "Object" type, a data type prior special to 

In [ ]:
# Convert title column to string type
df['title'] = df['title'].astype(str)

# Convert completed column to int type (0 for False, 1 for True)
df['completed'] = df['completed'].astype(int)
df.head()

To adhere to the principles of database normalization, particularly the Second Normal Form (2NF), I split the dataframe into two separate entities as according to 2NF, non-key attributes should be fully dependent on the entire primary key, avoiding partial dependencies within a table.

Upon examination of the original dataframe, where the assumed primary key is "id", it becomes apparent that the non-key attribute "userId" does not exhibit full dependency on the primary key. This is evident as different "id" values may correspond to the same "userId". Furthermore, there exists a partial dependency between the "title" and "completed" columns, where the uniqueness of their combination should be solely reliant on the primary key, but is currently influenced by the "userId".


The dataframe is split into 2 dataframe in the table structure as shown below:

Users Table:
* Columns: userId (Primary Key), 
* [The Other user-related information that might be add in later]

Tasks Table:
* Columns:
    * id (Primary Key)
    * userId (Foreign Key referencing Users table)
    * title
    * completed

In [ ]:
# first, reorganize the datarame into 2 different dataframe with the proper table structure
import pandas as pd

# Users Table
users_df = df[['userId']].drop_duplicates().reset_index(drop=True)
print(users_df.head())

# Tasks Table
tasks_df = df[['id', 'userId', 'title', 'completed']]
print(tasks_df.head())

### Part 3: Loading
1. load the data into a MySQL database.
2. Configure constraints like primary keys and foreign keys with their referencing table-column

In [ ]:
# load the dataframe as table into database
from sqlalchemy import create_engine
from config import settings

DATABASE_URL = f'mysql+mysqlconnector://{settings.db_user}:{settings.db_password}@{settings.db_host}:{settings.db_port}/{settings.db_name}'
engine = create_engine(DATABASE_URL)

# Load dataframe as table into MySQL database for Users
users_df.to_sql(name='Users', con=engine, if_exists='replace', index=False)
print("Users table created successfully.")
tasks_df.to_sql(name='Tasks', con=engine, if_exists='replace', index=False)
print("Tasks table created successfully.")

In [ ]:
# after loading the dataframe as table into the database, configure the primary keys and foreign keys of each table 
# so they can refer to each other

import mysql.connector

# Configure the connection to MySQL server
connection = mysql.connector.connect(
    host=settings.db_host,
    user=settings.db_user,
    password=settings.db_password,
    database=settings.db_name
)
cursor = connection.cursor()


# Define SQL query that we are using to set the primary keys and foreign keys
queries = ['ALTER TABLE Users ADD PRIMARY KEY (userId)',
         'ALTER TABLE Tasks ADD PRIMARY KEY (id)',
         'ALTER TABLE Tasks ADD CONSTRAINT fk_user_id FOREIGN KEY (userId) REFERENCES Users(userId)'
         ]
# Execute queries
for query in queries:
    cursor.execute(query)


cursor.close()
connection.close()


### Part 4: Data Modelling

1. For each table, display the table structure, the components are:
    * columns
    * primary keys
    * foreign key & its connected column at another table

2. Display a star schema of the data model as requested.
    * I drawn the star schema in "db_star_schema.drawio"

In [ ]:
from sqlalchemy import create_engine, inspect

DATABASE_URL = f'mysql+mysqlconnector://{settings.db_user}:{settings.db_password}@{settings.db_host}:{settings.db_port}/{settings.db_name}'
engine = create_engine(DATABASE_URL)
inspector = inspect(engine)

# Get a list of table names in the database
table_names = inspector.get_table_names()

# Iterate over each table to inspect its structure
for table_name in table_names:
    print(f"Table: {table_name}")
    
    # Get columns information for the table
    columns = inspector.get_columns(table_name)
    print("Columns:")
    for column in columns:
        print(f"- {column['name']}: {column['type']}")
    
    # Get primary key information for the table
    primary_key =inspector.get_pk_constraint(table_name)
    print("Primary Key:", primary_key)
    
    # Get foreign key information for the table
    foreign_keys = inspector.get_foreign_keys(table_name)
    print("Foreign Keys:")
    for foreign_key in foreign_keys:
        print(f"- {foreign_key['constrained_columns']} references {foreign_key['referred_table']}({foreign_key['referred_columns']})")
        
    print()


### Part 5: Data Visualization
for this part, I am going load the data from the database, one dataframe per table (Users and Tasks), then export them as csv for visualization using Excel.

In [ ]:
import pandas as pd
from sqlalchemy import create_engine

# Configure database connection
DATABASE_URL = f'mysql+mysqlconnector://{settings.db_user}:{settings.db_password}@{settings.db_host}:{settings.db_port}/{settings.db_name}'
engine = create_engine(DATABASE_URL)

# Load data from the database into dataframes, take a look at the first few rows to see if the data is fine
table1_df = pd.read_sql_table('Users', engine)
print(table1_df.head())
table2_df = pd.read_sql_table('Tasks', engine)
print(table2_df.head())

# Export dataframes to .csv files
table1_df.to_csv('Users.csv', index=False)
table2_df.to_csv('Tasks.csv', index=False)
